In [24]:
import pandas as pd
df = pd.read_excel('Vendas_simi.xlsx')

df['ID Produto'] = df['ID Produto'].astype(str)

In [25]:
# Criando uma matriz de utilidade
matriz_utilidade = df.pivot_table(index='ID Produto',
                                  columns='ID Cliente',
                                  values='Quantidade',
                                  fill_value=0)

matriz_utilidade.head()

ID Cliente,6,7,9,10,11,16,17,18,19,20,...,19116,19117,19118,19119,19120,19125,19126,19135,19137,19145
ID Produto,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
102,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
# Calculando a similaridade cosseno entre os users com base em seus padrões de nota
from sklearn.metrics.pairwise import cosine_similarity
matriz_similaridade = cosine_similarity(matriz_utilidade)

In [27]:
# Convertendo a matriz de similaridade em um DataFrame para fácil manipulação.
# Os índices e colunas são os nomes dos users.
df_similaridade = pd.DataFrame(matriz_similaridade, index=matriz_utilidade.index, columns=matriz_utilidade.index)

df_similaridade.head()

ID Produto,1,10,100,101,102,103,104,105,11,111,...,90,91,92,93,94,95,96,97,98,99
ID Produto,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.437258,0.340803,0.134030,0.236032,0.241814,0.339184,0.227494,0.465937,0.473790,...,0.503438,0.416404,0.522060,0.520051,0.371812,0.509163,0.281570,0.496907,0.086016,0.064321
10,0.437258,1.000000,0.037815,0.401587,0.231106,0.220695,0.138624,0.174518,0.726782,0.329571,...,0.228765,0.164182,0.207380,0.303384,0.319904,0.363531,0.441180,0.304647,0.058537,0.484631
100,0.340803,0.037815,1.000000,0.516547,0.248565,0.090958,0.157805,0.236698,0.028602,0.000000,...,0.341300,0.416344,0.461022,0.175331,0.224677,0.162433,0.163724,0.255372,0.000000,0.170743
101,0.134030,0.401587,0.516547,1.000000,0.217439,0.112746,0.095620,0.120197,0.157967,0.060749,...,0.281637,0.254812,0.278976,0.209989,0.162583,0.096931,0.290992,0.129680,0.000000,0.861828
102,0.236032,0.231106,0.248565,0.217439,1.000000,0.876460,0.417281,0.691731,0.147938,0.382551,...,0.418995,0.343460,0.519314,0.210872,0.171661,0.162076,0.487392,0.266033,0.512619,0.135212


In [38]:
def recomendar_produtos(produto_comprado, n_recomendacoes=3):
    if produto_comprado not in df_similaridade.index:
        return f"Produto {produto_comprado} não encontrado na base."

    # Pega a linha de similaridade para o produto
    similaridades = df_similaridade.loc[produto_comprado]

    # Remove o próprio produto da lista
    similaridades = similaridades.drop(produto_comprado)

    # Ordena pela similaridade decrescente e pega os top n
    recomendados = similaridades.sort_values(ascending=False).head(n_recomendacoes)

    return recomendados

# Exemplo de input
produto_usuario = input("Digite o ID do produto comprado: ")

# Gerar recomendação
recomendacoes = recomendar_produtos(produto_usuario)

# Verifica se é uma mensagem de erro
if isinstance(recomendacoes, str):
    print(recomendacoes)
else:
    # Formata os valores para porcentagem inteira
    recomendacoes_percent = (recomendacoes * 100).round(0).astype(int)

    print(f"Produtos recomendados para quem comprou o produto {produto_usuario}:")
    print("Produto, Similaridade")
    for produto_id, similaridade in recomendacoes_percent.items():
        print(f"  {produto_id}         {similaridade}%")

Digite o ID do produto comprado: 10
Produtos recomendados para quem comprou o produto 10:
Produto, Similaridade
  11         73%
  13         65%
  50         63%
